In this notebook, we will apply the information-matching method to find the optimal PMU placements that leads to full system observability.
The objective is translated to requiring non-singular FIM of the system.
In practice, we set the target FIM to be a diagonal matrix, where the diagonal element is set to be some small positive number.
By doing this, the eigenvalues of the optimal FIM are larger than this small number, and thus the FIM is non-singular.

Details of the calculation:
* Model: IEEE 39-bus system
* Candidate data: Voltage phasor at each bus
* Eigenvalue cutoff: 1e-5

In [ ]:
from pathlib import Path

import numpy as np
import cvxpy as cp
import matplotlib as mpl
import matplotlib.pyplot as plt

from information_matching.convex_optimization import ConvexOpt
from information_matching.utils import tol, eps

%matplotlib inline
plt.style.use("default")

In [ ]:
# Set directories
case = 39  # 39-bus case

WORK_DIR = Path().absolute()
FIM_DIR = WORK_DIR / "FIMs"
DATA_DIR = WORK_DIR / "models" / "data"

# Setup

In [ ]:
# Model information
nparams = 2 * case

## Get FIMs for candidate configurations

With some arguments, we might want to exclude some buses and preassigned PMUs on some buses.

For example, we can consider the formulation in https://ieeexplore.ieee.org/document/4519389, where the authors have some preassigned buses to place PMUs and exclude some buses as candidates.

In [ ]:
# No preassigned buses and include all buses as candidate locations
preassigned_buses = np.array([], dtype=int)
candidate_buses = np.arange(case) + 1

# # Some preassigned buses and few candidates from the paper
# # https://ieeexplore.ieee.org/document/4519389
# preassigned_buses = np.array([20, 23, 25, 29], dtype=int)
# candidate_buses = np.array(
#     [2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15,
#      16, 17, 18, 19, 21, 22, 24, 26, 27, 28, 39,]
#     dtype=int,
# )

In [ ]:
# Finalize the candidate buses after considering preassigned buses
candidate_buses=np.sort(np.unique(np.append(preassigned_buses,candidate_buses)))

# Configurations
nconfigs = len(candidate_buses)
configs = candidate_buses
config_ids = np.array([f"bus{ii}" for ii in configs])
config_ids

In [ ]:
# Load configuration FIMs
fim_configs_tensor = np.empty((nconfigs, nparams, nparams))
for ii, bus in enumerate(configs):
    ff = np.loadtxt(FIM_DIR / f"fim_bus{bus}.csv", delimiter=",")
    fim_configs_tensor[ii] = ff

## Get the target FIM

Like in the IEEE 14-bus system case, we will also set the target FIM to be a diagonal matrix with a positive cutoff value for the eigenvalues on the diagonal.
However, one thing that is different is that we might have preassigned buses in this case.
To incorporate this information, we will subtract the FIMs from the preassigned buses (with weight 1.0) from the target FIM.
That is, the effective target FIM is the diagonal matrix minus the FIMs from the preassigned buses.

In [ ]:
# Load the FIM of the preassigned buses and sum them up
fim_preassigned = np.zeros((nparams, nparams))
for bus in preassigned_buses:
    ff = np.loadtxt(DATA_DIR / "FIMs" / f"fim_bus{bus}.csv", delimiter=",")
    fim_preassigned += ff

In [ ]:
# Matrix to impose tolerance for the non-zero eigenvalues
lambda_tol = 1e-5
fim_tol = lambda_tol * np.eye(nparams)
print("Eigenvalue lower bound:", lambda_tol)

In [ ]:
# Effective target FIM
# Constant matrix on the RHS of the PSD constraint
fim_target = fim_tol - fim_preassigned

# Convex optimization

Additional note:

We will perform the optimization iteratively.
In each iteration, we scale the weights being optimize by the optimal weights from the previous iteration.
By doing so, the objective function in the convex optimization better mimics $\ell_0$-norm to enforce sparsity.

In [ ]:
# Construct the input FIMs
# FIM target is fine, because we only apply scale 1.0
# FIM configs
fim_configs = {}
for ii, identifier in enumerate(config_ids):
    fim_configs.update({identifier: {"fim": fim_configs_tensor[ii]}})

In [ ]:
# Convex optimization
# Settings
cvx_tol = eps ** 0.75
solver = dict(
    verbose=False, solver=cp.SDPA, epsilonStar=cvx_tol, gammaStar=0.5, lambdaStar=1e3
)
print("Tolerance:", cvx_tol)

plt.figure()
wopt = np.ones(nconfigs)  # Initial weights
for jj in range(50):
    # Update the weight scales
    for ii, identifier in enumerate(config_ids):
        # Add weight scale information,aAdd some small number for numerical stability
        fim_configs[identifier].update({"weight_scale": 1 / (wopt[ii] + 1e-15)})
    cvxopt = ConvexOpt(fim_target, fim_configs, l1norm_obj=True)

    try:
        # Solve
        cvxopt.solve(solver=solver)
        result = cvxopt.result.copy()
        wopt = result["wm"]
        dual = result["dual_wm"]
        print("Violation:", cvxopt.constraints[1].violation())
    except Exception:
        cvxopt.result = result
        break

    plt.plot(wopt, label=jj)

plt.yscale("log")
plt.xticks(range(nconfigs), config_ids, rotation=90)
plt.ylabel("Weights")
plt.legend(title="Iteration", bbox_to_anchor=(1, 1))
plt.show()

## Post-processing

In [ ]:
# # Plot the weights of the last step
# plt.figure()
# plt.title(f"Eigenvalue lower bound: {lambda_tol:0.3e}")
# plt.plot(wopt, label="weights")
# plt.plot(dual, label="dual weights")
# plt.yscale("log")
# plt.xticks(range(nconfigs), config_ids, rotation=90)
# plt.legend()
# plt.show()

In [ ]:
# Get the optimal buses
idx_wopt = np.where(wopt > cvx_tol)[0]  # Index to the optimal buses
print("Optimal buses:")
_ = [print(f'{config_ids[ii]} \t {wopt[ii]}') for ii in idx_wopt]

In [ ]:
# Test the optimal buses
weighted_fims_configs = fim_configs_tensor * wopt.reshape((-1, 1, 1))
I = np.sum(weighted_fims_configs[idx_wopt], axis=0)
print("Eigenvalues:")
print(np.linalg.eigvalsh(I))

plt.figure()
cbound = np.max([abs(np.min(I)), abs(np.max(I))])
plt.imshow(I, cmap="bwr", norm=mpl.colors.SymLogNorm(1e0, vmin=-cbound, vmax=cbound))
plt.colorbar()
plt.show()

Even though the smallest eigenvalue of the optimal FIM is smaller than the cutoff, but they are pretty close.
We believe this is just numerical artifact.